In [ ]:
%matplotlib notebook

import multiprocessing.pool
import pathlib
import time

import matplotlib.pyplot as plt
import torch
import tqdm

from laueimproc import Diagram
from laueimproc.io.download import get_samples  # gives access to the dataset
from laueimproc.opti.manager import DiagramManager

In [ ]:
manager = DiagramManager()
manager.verbose = True  # to display the operations in background
manager.max_mem_percent = 75  # trigger cache clear after 75% of ram full, default based on swapiness
manager.update()  # dereference unreachable diagrams

In [ ]:
all_files = sorted(get_samples().iterdir())*100
diagrams = [Diagram(f) for f in all_files]
for diagram in diagrams:
    diagram.find_spots()

In [ ]:
"""Test compression ROIS"""

from laueimproc.opti.comp import compress_rois, decompress_rois
from laueimproc.opti.cache import getsizeof

brut = comp = 0
t_start = time.time()
for diagram in tqdm.tqdm(diagrams):
    rois_comp = compress_rois(diagram.rois)
    brut += getsizeof(diagram.rois)
    comp += getsizeof(rois_comp)
    # print(f"avant={getsizeof(diagram.rois)}, apres={getsizeof(rois_comp)}")
    assert torch.max(torch.abs(diagram.rois - decompress_rois(rois_comp))).item() <= 1/131070
print((time.time()-t_start)/len(diagrams))
print(f"total {brut//1000} to {comp//1000} (ratio {brut/comp:.1f})")

In [ ]:
"""Test memory limit"""

for diagram in tqdm.tqdm(diagrams, smoothing=0.01):
    intensities = diagram.compute_pxl_intensities()
    sorted_indexs = torch.argsort(intensities, descending=True)
    diagram.filter_spots(sorted_indexs, msg="sorted by intensities", inplace=True)